#**Task Overview**#

The goal of this task is to detect deception in dialogues by training a logistic regression model using stochastic gradient descent (SGD). The dataset consists of text messages from a game where each message is labeled as truthful or deceptive.

##**Two text vectorization methods are used:**##

CountVectorizer (word frequency-based representation)

TF-IDF Vectorizer (importance-based representation)

**The model is trained separately for both vectorization methods, and their performance is compared.**


#**Installation of required files**#

In [1]:
# First install any required packages that aren't pre-installed in Colab
!pip install jsonlines spacy scikit-learn gdown

#**Importing necessary libraries**#

In [2]:
import jsonlines
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import SGDClassifier
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import log_loss, accuracy_score, f1_score
from scipy.sparse import csr_matrix
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
import os
import gdown
import warnings
warnings.filterwarnings("ignore")

#**Dataset and Configuration**#
The dataset is stored in JSONL format and is downloaded from Google Drive if not available.

The dataset contains game dialogues, where players communicate with each other.

Each message is labeled as truthful or deceptive (sender_annotation or receiver_annotation).

The task can be configured to focus on either:

**SENDER**: Whether the sender is truthful or deceptive.

**RECEIVER**: Whether the receiver perceives the message as truthful or deceptive.

The "**POWER**" setting determines whether the model considers game score changes as additional features.

In [3]:
TASK = "SENDER"  # Can be "SENDER" or "RECEIVER"
POWER = "y"      # Can be "y" (include power) or "n" (exclude power)

# File IDs extracted from Google Drive links
FILE_IDS = {
    'train.jsonl': '1sHVikR018z4QEBmHPUVwRXoKmgv7G69q',
    'validation.jsonl': '1Y-MKTjzrrP8X8oTxB8yz9awilRsXSNS_',
    'test.jsonl': '1sDgZ98_OXsoCt5WxroXLSlwMcCUZp4zm'
}

DATA_DIR = 'data/'

print(f"Configuration: TASK={TASK}, POWER={POWER}")
def download_files():
    # Create data directory if it doesn't exist
    os.makedirs(DATA_DIR, exist_ok=True)

    # Download each file if it doesn't already exist
    for filename, file_id in FILE_IDS.items():
        filepath = os.path.join(DATA_DIR, filename)
        if not os.path.exists(filepath):
            url = f'https://drive.google.com/uc?id={file_id}'
            gdown.download(url, filepath, quiet=False)
            print(f"Downloaded {filename}")
        else:
            print(f"{filename} already exists, skipping download")

# Download the files
download_files()

Configuration: TASK=SENDER, POWER=y


Downloading...
From: https://drive.google.com/uc?id=1sHVikR018z4QEBmHPUVwRXoKmgv7G69q
To: /content/data/train.jsonl
100%|██████████| 2.47M/2.47M [00:00<00:00, 18.9MB/s]


Downloaded train.jsonl


Downloading...
From: https://drive.google.com/uc?id=1Y-MKTjzrrP8X8oTxB8yz9awilRsXSNS_
To: /content/data/validation.jsonl
100%|██████████| 246k/246k [00:00<00:00, 4.71MB/s]


Downloaded validation.jsonl


Downloading...
From: https://drive.google.com/uc?id=1sDgZ98_OXsoCt5WxroXLSlwMcCUZp4zm
To: /content/data/test.jsonl
100%|██████████| 490k/490k [00:00<00:00, 6.69MB/s]

Downloaded test.jsonl


BERT Transformer implementation

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from transformers import BertTokenizer, BertModel, AutoModel
from torch.utils.data import Dataset, DataLoader
import numpy as np
import json
from typing import List, Dict, Optional
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
from tqdm import tqdm
from torch.cuda.amp import GradScaler, autocast

class DiplomacyDataset(Dataset):
    def __init__(self, filepath: str, tokenizer: BertTokenizer, max_len: int = 64):
        self.instances = []
        self.tokenizer = tokenizer
        with open(filepath, 'r') as f:
            for line in f:
                data = json.loads(line)
                if len(data['messages']) == 0:
                    continue

                msgs, labels = [], []
                for msg, label in zip(data['messages'], data['sender_labels']):
                    if str(label).lower() not in ['true', 'false']:
                        continue
                    msg = msg.lower().strip()
                    msgs.append(msg)
                    labels.append(1 if str(label).lower() == 'true' else 0)

                if msgs:
                    self.instances.append((msgs, labels))

        self.max_len = max_len

    def __len__(self):
        return len(self.instances)

    def __getitem__(self, idx):
        messages, labels = self.instances[idx]
        # Process only up to 10 messages per instance to save memory
        messages = messages[:10]
        labels = labels[:10]

        encoded = [self.tokenizer(
            msg,
            padding='max_length',
            truncation=True,
            max_length=self.max_len,
            return_tensors='pt'
        ) for msg in messages]

        input_ids = torch.cat([e['input_ids'] for e in encoded], dim=0)
        attention_mask = torch.cat([e['attention_mask'] for e in encoded], dim=0)
        labels = torch.tensor(labels, dtype=torch.long)
        return input_ids, attention_mask, labels

class BertPooler(nn.Module):
    def __init__(self, model_name='bert-base-uncased', trainable=True):
        super().__init__()
        # Use gradient checkpointing to save memory
        self.bert = AutoModel.from_pretrained(model_name)
        if trainable:
            self.bert.gradient_checkpointing_enable()
        for p in self.bert.parameters():
            p.requires_grad = trainable
        self.hidden_size = self.bert.config.hidden_size

    def forward(self, input_ids, attention_mask):
        with torch.cuda.amp.autocast():
            output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return output.last_hidden_state[:, 0]

class HierarchicalLSTM(nn.Module):
    def __init__(self, pos_weight: float = 1.0, dropout=0.3):
        super().__init__()
        self.bert_pooler = BertPooler()

        # Smaller LSTM to save memory
        self.lstm = nn.LSTM(
            input_size=self.bert_pooler.hidden_size,
            hidden_size=128,  # Reduced from 256
            num_layers=1,    # Reduced from 2
            bidirectional=True,
            batch_first=True,
            dropout=0
        )

        self.dropout = nn.Dropout(dropout)

        self.classifier = nn.Sequential(
            nn.Linear(128 * 2, 64),  # Reduced dimensions
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(64, 2)
        )

        self._init_weights()
        self.pos_weight = torch.tensor([1.0, pos_weight])

    def _init_weights(self):
        for name, module in self.named_modules():
            if isinstance(module, nn.Linear):
                nn.init.xavier_normal_(module.weight)
                if module.bias is not None:
                    nn.init.constant_(module.bias, 0)
            elif isinstance(module, nn.LSTM):
                for name, param in module.named_parameters():
                    if 'weight' in name:
                        nn.init.orthogonal_(param)
                    elif 'bias' in name:
                        nn.init.constant_(param, 0)
                        n = param.size(0)
                        param.data[n//4:n//2].fill_(1.0)

    def forward(self, input_ids, attention_mask, labels=None):
        B, M, T = input_ids.shape
        input_ids = input_ids.view(B * M, T)
        attention_mask = attention_mask.view(B * M, T)

        message_vecs = self.bert_pooler(input_ids, attention_mask)
        message_vecs = self.dropout(message_vecs).view(B, M, -1)

        lstm_out, _ = self.lstm(message_vecs)
        lstm_out = self.dropout(lstm_out)

        logits = self.classifier(lstm_out)

        output = {'logits': logits}
        if labels is not None:
            logits_flat = logits.view(-1, 2)
            labels_flat = labels.view(-1)
            mask = labels_flat != -1
            loss = F.cross_entropy(
                logits_flat[mask],
                labels_flat[mask],
                weight=self.pos_weight.to(logits.device),
                label_smoothing=0.1
            )
            output['loss'] = loss

        return output

def collate_batch(batch):
    max_messages = min(10, max([x[0].size(0) for x in batch]))  # Cap at 10 messages
    B = len(batch)
    T = batch[0][0].size(1)

    input_ids = torch.zeros(B, max_messages, T, dtype=torch.long)
    attention_mask = torch.zeros_like(input_ids)
    labels = torch.full((B, max_messages), -1, dtype=torch.long)

    for i, (ids, mask, lbl) in enumerate(batch):
        m = min(ids.size(0), max_messages)
        input_ids[i, :m] = ids[:m]
        attention_mask[i, :m] = mask[:m]
        labels[i, :m] = lbl[:m]

    return input_ids, attention_mask, labels

def train_model(train_file, val_file, epochs=10, batch_size=4, pos_weight=15):  # Reduced batch_size
    torch.cuda.empty_cache()

    tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
    train_data = DiplomacyDataset(train_file, tokenizer)
    val_data = DiplomacyDataset(val_file, tokenizer)

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = HierarchicalLSTM(pos_weight=pos_weight).to(device)

    # Freeze first few BERT layers to save memory
    for param in list(model.bert_pooler.bert.encoder.layer[:6].parameters()):
        param.requires_grad = False

    optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5, weight_decay=0.01)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=2, verbose=True
    )

    best_f1 = 0
    patience = 3
    no_improve = 0

    scaler = GradScaler()
    accumulation_steps = max(1, 32 // batch_size)  # Increased accumulation steps

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        optimizer.zero_grad()

        for i, (input_ids, attention_mask, labels) in enumerate(tqdm(train_loader)):
            input_ids = input_ids.to(device, non_blocking=True)
            attention_mask = attention_mask.to(device, non_blocking=True)
            labels = labels.to(device, non_blocking=True)

            with autocast():
                output = model(input_ids, attention_mask, labels)
                loss = output['loss'] / accumulation_steps

            scaler.scale(loss).backward()

            if (i + 1) % accumulation_steps == 0:
                scaler.unscale_(optimizer)
                torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
                scaler.step(optimizer)
                scaler.update()
                optimizer.zero_grad()
                torch.cuda.empty_cache()

            total_loss += loss.item() * accumulation_steps

        # Evaluation
        model.eval()
        preds, trues = [], []
        with torch.no_grad():
            for input_ids, attention_mask, labels in val_loader:
                input_ids = input_ids.to(device, non_blocking=True)
                attention_mask = attention_mask.to(device, non_blocking=True)

                with autocast():
                    output = model(input_ids, attention_mask)

                logits = output['logits']
                pred = torch.argmax(logits, dim=-1).cpu().numpy()
                labels = labels.numpy()

                for p, l in zip(pred, labels):
                    for pi, li in zip(p, l):
                        if li != -1:
                            preds.append(pi)
                            trues.append(li)

        f1 = f1_score(trues, preds, average='macro')
        accuracy = accuracy_score(trues, preds)
        precision = precision_score(trues, preds)
        recall = recall_score(trues, preds)

        print(f"[Epoch {epoch+1}] Train Loss: {total_loss/len(train_loader):.4f}")
        print(f"Validation - F1: {f1:.4f}, Acc: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

        scheduler.step(f1)

        if f1 > best_f1:
            best_f1 = f1
            no_improve = 0
            torch.save(model.state_dict(), 'best_model.pt')
            print("New best model saved!")
        else:
            no_improve += 1
            if no_improve >= patience:
                print(f"No improvement for {patience} epochs, stopping early")
                break

    model.load_state_dict(torch.load('best_model.pt'))
    return model

if __name__ == "__main__":
    # Set environment variable to help with memory fragmentation
    import os
    os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

    trained_model = train_model(
        train_file="data/train.jsonl",
        val_file="data/validation.jsonl",
        epochs=15,
        batch_size=1,  # Reduced from 8
        pos_weight=15
    )

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
<ipython-input-1-35665d26aab5>:187: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
  0%|          | 0/184 [00:00<?, ?it/s]<ipython-input-1-35665d26aab5>:200: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
<ipython-input-1-35665d26aab5>:70: FutureWarning: `torch.cuda.amp.autocast(arg

[Epoch 1] Train Loss: 0.1983
Validation - F1: 0.4804, Acc: 0.9246, Precision: 0.9246, Recall: 1.0000
New best model saved!


  0%|          | 0/184 [00:00<?, ?it/s]<ipython-input-1-35665d26aab5>:200: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
<ipython-input-1-35665d26aab5>:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 184/184 [00:11<00:00, 16.72it/s]
<ipython-input-1-35665d26aab5>:224: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Epoch 2] Train Loss: 0.0779
Validation - F1: 0.4804, Acc: 0.9246, Precision: 0.9246, Recall: 1.0000


  0%|          | 0/184 [00:00<?, ?it/s]<ipython-input-1-35665d26aab5>:200: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
<ipython-input-1-35665d26aab5>:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 184/184 [00:10<00:00, 17.00it/s]
<ipython-input-1-35665d26aab5>:224: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Epoch 3] Train Loss: 0.0788
Validation - F1: 0.4804, Acc: 0.9246, Precision: 0.9246, Recall: 1.0000


  0%|          | 0/184 [00:00<?, ?it/s]<ipython-input-1-35665d26aab5>:200: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():
<ipython-input-1-35665d26aab5>:70: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
100%|██████████| 184/184 [00:10<00:00, 16.90it/s]
<ipython-input-1-35665d26aab5>:224: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


[Epoch 4] Train Loss: 0.0726
Validation - F1: 0.4804, Acc: 0.9246, Precision: 0.9246, Recall: 1.0000
No improvement for 3 epochs, stopping early
